### $\S$ 1. CAP理论和分布式数据库管理系统

#### 一. CAP理论介绍  
1. CAP理论:  
一个分布式系统, 只能确保3种属性中的2种:   
    * Consistency: 一致性
    * Availability: 可用性
    * Partition Tolerance: 分区容错性
    
2. CAP理论在大数据中是很重要的理论, 经常要在三者之中权衡

#### 二. Partition Tolerance: 分区容错性
1. 这个条件表明: 在如下条件下, 系统可以持续运行: 
    * 一个系统可以忍受任意数量的网络失败, 而不会造成整个网络的失败
    * 即使经过断断续续的停电故障, 数据记录仍能在节点和网络间被复制到足够的份数, 来保证系统正常运行
2. 分区容错性是现代分布式系统必须的属性, 因此只能在一致性和可用性之间权衡
    <img src="./img/partitiontolerance.png" height="70%" width="70%">

#### 三. High Consistency: 高一致性
1. 这个条件表明: 所有节点在同一时间看到同一份数据  
 简单说就是: 一个读操作, 将返回所有节点上最近一次写操作更新的值. 即所有节点都返回相同的,最近一次操作的值
2. 一个系统是一致性的: 如果一项事务从一致性状态开始, 又以一致性状态结束   
3. 一致性的系统, 在事务执行过程中可能会产生非一致性的状态. 但只要事务在任一阶段发生错误, 整个事务都会回滚
4. 图示  
 如下图, 有两条不同的数据(“Bulbasaur” and “Pikachu”)对应不同的时间戳. 第三部分的输出是“Pikachu”(最后的输入)  
 但是节点需要时间更新这个最新值, 而此时网络往往不可用.
  <img src="./img/HighConsistency.png" height="70%" width="70%">
  
#### 四. High Availability: 高可用性
1. 这个条件表明: 系统在任何时间都保有100%的可操作性
2. 这种模式下的数据库是与时间无关的  
 如下图, 我们不知道“Pikachu”和“Bulbasaur”哪一条记录被先添加.
 因此, 输出可能是“Pikachu” or “Bulbasaur” . 所以"High Availability"的系统不适合处理高频率的流式数据
    <img src="./img/highavaliable.png" height="70%" width="70%">